<a href="https://colab.research.google.com/github/ElkinSn/Coursera_Capstone/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing libraries**

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


**Downloading files and placing them on the server**

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


**Tranforming the data into a pandas dataframe**

In [4]:
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


**Using geopy library to get the latitude and longitude values of New York City.**

In [6]:
address = 'New York City, NY'
# We need to define a user_agent. The name of our agent is ny_explorer.
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


**Creating a map of New York with neighborhoods**

In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [8]:
import folium
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Integrating the information of FOURSQUARE venues**

In [9]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [13]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'QSQC42XYF50MXN2PNVIVQ5ASUGUY50RXSBSBCNBNEADVIXYB'
CLIENT_SECRET = 'JCUMNSDJ1YDPTLQCRI0DZ4GUFHCK1V2FBANA0TM50I1X5XWE'
VERSION = '20181020'

**>> Looking for Colombian Restaurants**

In [14]:
#https://developer.foursquare.com/docs/resources/categories
#Colombian Restaurant: 58daa1558bbb0b01f18ec1f4
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_colombian = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='58daa1558bbb0b01f18ec1f4')
newyork_venues_colombian.head()

https://api.foursquare.com/v2/venues/search?&client_id=QSQC42XYF50MXN2PNVIVQ5ASUGUY50RXSBSBCNBNEADVIXYB&client_secret=JCUMNSDJ1YDPTLQCRI0DZ4GUFHCK1V2FBANA0TM50I1X5XWE&v=20181020&ll=40.75665808227519,-74.00011136202637&radius=1000&limit=500&categoryId=58daa1558bbb0b01f18ec1f4
{'meta': {'code': 200, 'requestId': '6004a6ecba07166a6ac3254e'}, 'response': {'venues': []}}
[]
Empty DataFrame
Columns: []
Index: []


""


**>> Looking for Latino Restaurants**

In [40]:
#https://developer.foursquare.com/docs/resources/categories
#Latino Restaurant: 4bf58dd8d48988d1be941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_colombian = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1be941735')
newyork_venues_colombian.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Hola España,40.879160,-73.904553,Latin American Restaurant
1,Marble Hill,40.876551,-73.91066,La Caridad,40.869219,-73.903219,Latin American Restaurant
2,Marble Hill,40.876551,-73.91066,Leche y Miel,40.883742,-73.901857,Latin American Restaurant
3,Marble Hill,40.876551,-73.91066,La Tia,40.878808,-73.905187,Latin American Restaurant
4,Marble Hill,40.876551,-73.91066,Empanadas Monumental,40.883476,-73.901964,Empanada Restaurant


In [41]:
newyork_venues_colombian.shape

(934, 7)

**Generating competitors map**

In [42]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [43]:
map_newyork_colombian = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_colombian, 'red', map_newyork_colombian)

map_newyork_colombian

**Grouping the information by neighborhoods**

In [44]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [45]:
manhattan_grouped = newyork_venues_colombian.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_colombian['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,9,9,9,9,9,9
Carnegie Hill,11,11,11,11,11,11
Central Harlem,9,9,9,9,9,9
Chelsea,23,23,23,23,23,23
Chinatown,39,39,39,39,39,39
Civic Center,18,18,18,18,18,18
Clinton,47,47,47,47,47,47
East Harlem,17,17,17,17,17,17
East Village,40,40,40,40,40,40


**Analyzing by neighborhood**

In [46]:
manhattan_onehot = pd.get_dummies(newyork_venues_colombian[['Venue Category']], prefix="", prefix_sep="")
manhattan_onehot['Neighborhood'] = newyork_venues_colombian['Neighborhood'] 
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_onehot.head()

,Neighborhood,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Bakery,Bar,Brazilian Restaurant,Café,Caribbean Restaurant,Churrascaria,Cuban Restaurant,Deli / Bodega,Empanada Restaurant,Food Truck,French Restaurant,Italian Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Mexican Restaurant,Middle Eastern Restaurant,Nightclub,Office,Peruvian Restaurant,Pizza Place,Puerto Rican Restaurant,Seafood Restaurant,South American Restaurant,Spanish Restaurant,Steakhouse,Tapas Restaurant,Venezuelan Restaurant,Wine Bar
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [47]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Bakery,Bar,Brazilian Restaurant,Café,Caribbean Restaurant,Churrascaria,Cuban Restaurant,Deli / Bodega,Empanada Restaurant,Food Truck,French Restaurant,Italian Restaurant,Jazz Club,Juice Bar,Latin American Restaurant,Lounge,Mexican Restaurant,Middle Eastern Restaurant,Nightclub,Office,Peruvian Restaurant,Pizza Place,Puerto Rican Restaurant,Seafood Restaurant,South American Restaurant,Spanish Restaurant,Steakhouse,Tapas Restaurant,Venezuelan Restaurant,Wine Bar
0,Battery Park City,0.111111,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000
1,Carnegie Hill,0.090909,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.636364,0.000000,0.000000,0.000000,0.000000,0.000000,0.181818,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.086957,0.130435,0.000000,0.043478,0.000000,0.043478,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.347826,0.000000,0.043478,0.000000,0.000000,0.000000,0.130435,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000
4,Chinatown,0.076923,0.102564,0.000000,0.000000,0.000000,0.051282,0.000000,0.000000,0.000000,0.000000,0.025641,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.435897,0.000000,0.025641,0.000000,0.000000,0.000000,0.153846,0.00,0.000000,0.000000,0.051282,0.000000,0.000000,0.025641,0.025641,0.000000
5,Civic Center,0.222222,0.166667,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.055556,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Clinton,0.042553,0.085106,0.000000,0.000000,0.000000,0.170213,0.000000,0.000000,0.000000,0.000000,0.000000,0.106383,0.021277,0.000000,0.000000,0.000000,0.000000,0.319149,0.000000,0.021277,0.000000,0.000000,0.021277,0.127660,0.00,0.000000,0.000000,0.063830,0.000000,0.021277,0.000000,0.000000,0.000000
7,East Harlem,0.058824,0.058824,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.588235,0.000000,0.058824,0.000000,0.000000,0.000000,0.058824,0.00,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.050000,0.075000,0.000000,0.000000,0.000000,0.075000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.025000,0.000000,0.000000,0.000000,0.000000,0.450000,0.000000,0.050000,0.000000,0.025000,0.000000,0.050000,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.050000,0.025000,0.025000
9,Financial District,0.000000,0.125000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.125000,0.000000,0.000000,0.125000,0.00,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000


In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**>> Raking the venues**

In [49]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Latin American Restaurant,Arepa Restaurant,Brazilian Restaurant,Empanada Restaurant,Middle Eastern Restaurant,Peruvian Restaurant,South American Restaurant,Argentinian Restaurant,Bar,Café
1,Carnegie Hill,Latin American Restaurant,Peruvian Restaurant,Arepa Restaurant,Argentinian Restaurant,Food Truck,Empanada Restaurant,Deli / Bodega,Cuban Restaurant,Churrascaria,Caribbean Restaurant
2,Central Harlem,Latin American Restaurant,Food Truck,Spanish Restaurant,Empanada Restaurant,Wine Bar,Caribbean Restaurant,Deli / Bodega,Cuban Restaurant,Churrascaria,Café
3,Chelsea,Latin American Restaurant,Peruvian Restaurant,Argentinian Restaurant,Arepa Restaurant,Bakery,Brazilian Restaurant,French Restaurant,Caribbean Restaurant,Cuban Restaurant,Mexican Restaurant
4,Chinatown,Latin American Restaurant,Peruvian Restaurant,Argentinian Restaurant,Arepa Restaurant,South American Restaurant,Brazilian Restaurant,Mexican Restaurant,Tapas Restaurant,Venezuelan Restaurant,Deli / Bodega


**Clustering the neighborhoods**

In [50]:
kclusters = 10
manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)
kmeans.labels_[0:10]

array([9, 7, 8, 6, 1, 4, 3, 7, 1, 9], dtype=int32)

**>> Generating a new table**

In [51]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Latin American Restaurant,Tapas Restaurant,Lounge,Empanada Restaurant,Wine Bar,Churrascaria,Food Truck,Deli / Bodega,Cuban Restaurant,Caribbean Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,1,Latin American Restaurant,Peruvian Restaurant,Argentinian Restaurant,Arepa Restaurant,South American Restaurant,Brazilian Restaurant,Mexican Restaurant,Tapas Restaurant,Venezuelan Restaurant,Deli / Bodega
2,Manhattan,Washington Heights,40.851903,-73.936900,8,Latin American Restaurant,Empanada Restaurant,South American Restaurant,Seafood Restaurant,Caribbean Restaurant,Arepa Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Bakery
3,Manhattan,Inwood,40.867684,-73.921210,2,Latin American Restaurant,Venezuelan Restaurant,Spanish Restaurant,Lounge,Empanada Restaurant,Wine Bar,Churrascaria,Deli / Bodega,Cuban Restaurant,Caribbean Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Latin American Restaurant,Empanada Restaurant,Wine Bar,Churrascaria,French Restaurant,Food Truck,Deli / Bodega,Cuban Restaurant,Caribbean Restaurant,Jazz Club


**>> Generating clusters' map**

In [52]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**>> Generating tables by cluster**

* CLUSTER 0

In [53]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,South American Restaurant,Peruvian Restaurant,Latin American Restaurant,Brazilian Restaurant,Spanish Restaurant,Caribbean Restaurant,Empanada Restaurant,Deli / Bodega,Cuban Restaurant,Churrascaria


* CLUSTER 1

In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Latin American Restaurant,Peruvian Restaurant,Argentinian Restaurant,Arepa Restaurant,South American Restaurant,Brazilian Restaurant,Mexican Restaurant,Tapas Restaurant,Venezuelan Restaurant,Deli / Bodega
19,East Village,Latin American Restaurant,Argentinian Restaurant,Brazilian Restaurant,Arepa Restaurant,Mexican Restaurant,Tapas Restaurant,South American Restaurant,Empanada Restaurant,Peruvian Restaurant,Food Truck
20,Lower East Side,Latin American Restaurant,Peruvian Restaurant,Argentinian Restaurant,Empanada Restaurant,South American Restaurant,Brazilian Restaurant,Tapas Restaurant,Mexican Restaurant,Venezuelan Restaurant,Deli / Bodega
25,Manhattan Valley,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Argentinian Restaurant,Brazilian Restaurant,Café,Churrascaria,Food Truck,Empanada Restaurant,Deli / Bodega
31,Noho,Latin American Restaurant,Peruvian Restaurant,Argentinian Restaurant,Empanada Restaurant,Arepa Restaurant,Tapas Restaurant,Brazilian Restaurant,South American Restaurant,Mexican Restaurant,Venezuelan Restaurant
33,Midtown South,Latin American Restaurant,Brazilian Restaurant,South American Restaurant,Arepa Restaurant,Empanada Restaurant,Peruvian Restaurant,Argentinian Restaurant,Food Truck,Office,Caribbean Restaurant
35,Turtle Bay,Latin American Restaurant,Empanada Restaurant,Peruvian Restaurant,Seafood Restaurant,Food Truck,Brazilian Restaurant,Jazz Club,Argentinian Restaurant,Bar,Café
36,Tudor City,Latin American Restaurant,Empanada Restaurant,Argentinian Restaurant,Food Truck,Brazilian Restaurant,Peruvian Restaurant,Wine Bar,Churrascaria,Deli / Bodega,Cuban Restaurant
37,Stuyvesant Town,Latin American Restaurant,Argentinian Restaurant,Wine Bar,Brazilian Restaurant,Food Truck,Deli / Bodega,Peruvian Restaurant,Tapas Restaurant,Bar,Café
39,Hudson Yards,Latin American Restaurant,Empanada Restaurant,Peruvian Restaurant,Brazilian Restaurant,Argentinian Restaurant,Office,French Restaurant,Arepa Restaurant,Steakhouse,Café


* CLUSTER 2

In [55]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Latin American Restaurant,Tapas Restaurant,Lounge,Empanada Restaurant,Wine Bar,Churrascaria,Food Truck,Deli / Bodega,Cuban Restaurant,Caribbean Restaurant
3,Inwood,Latin American Restaurant,Venezuelan Restaurant,Spanish Restaurant,Lounge,Empanada Restaurant,Wine Bar,Churrascaria,Deli / Bodega,Cuban Restaurant,Caribbean Restaurant
4,Hamilton Heights,Latin American Restaurant,Empanada Restaurant,Wine Bar,Churrascaria,French Restaurant,Food Truck,Deli / Bodega,Cuban Restaurant,Caribbean Restaurant,Jazz Club
26,Morningside Heights,Latin American Restaurant,Peruvian Restaurant,Argentinian Restaurant,Wine Bar,Food Truck,Empanada Restaurant,Deli / Bodega,Cuban Restaurant,Churrascaria,Caribbean Restaurant


* CLUSTER 3

In [56]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Brazilian Restaurant,Peruvian Restaurant,Latin American Restaurant,Argentinian Restaurant,Empanada Restaurant,Mexican Restaurant,Pizza Place,Arepa Restaurant,South American Restaurant,Bar
14,Clinton,Latin American Restaurant,Brazilian Restaurant,Peruvian Restaurant,Empanada Restaurant,Argentinian Restaurant,South American Restaurant,Arepa Restaurant,Steakhouse,Food Truck,Office
15,Midtown,Latin American Restaurant,Brazilian Restaurant,Empanada Restaurant,South American Restaurant,Arepa Restaurant,Food Truck,Argentinian Restaurant,Peruvian Restaurant,Steakhouse,Office
16,Murray Hill,Latin American Restaurant,Brazilian Restaurant,Empanada Restaurant,Food Truck,South American Restaurant,Argentinian Restaurant,Arepa Restaurant,Deli / Bodega,Peruvian Restaurant,Italian Restaurant


* CLUSTER 4

In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Greenwich Village,Latin American Restaurant,Peruvian Restaurant,Arepa Restaurant,Brazilian Restaurant,Argentinian Restaurant,Mexican Restaurant,Empanada Restaurant,Spanish Restaurant,South American Restaurant,Nightclub
21,Tribeca,Peruvian Restaurant,Latin American Restaurant,Arepa Restaurant,Brazilian Restaurant,Argentinian Restaurant,Spanish Restaurant,Asian Restaurant,Bakery,Bar,Café
23,Soho,Latin American Restaurant,Peruvian Restaurant,Arepa Restaurant,Argentinian Restaurant,Brazilian Restaurant,Spanish Restaurant,Mexican Restaurant,Tapas Restaurant,Empanada Restaurant,Asian Restaurant
24,West Village,Peruvian Restaurant,Brazilian Restaurant,Latin American Restaurant,Arepa Restaurant,Spanish Restaurant,Bakery,Nightclub,Caribbean Restaurant,Mexican Restaurant,Tapas Restaurant
32,Civic Center,Latin American Restaurant,Arepa Restaurant,Peruvian Restaurant,Argentinian Restaurant,Brazilian Restaurant,Middle Eastern Restaurant,Asian Restaurant,Bakery,Bar,Café


* CLUSTER 5

In [58]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 5, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Latin American Restaurant,Juice Bar,Argentinian Restaurant,Arepa Restaurant,Spanish Restaurant,French Restaurant,Tapas Restaurant,Asian Restaurant,Bakery,Bar
10,Lenox Hill,Latin American Restaurant,Juice Bar,Empanada Restaurant,Arepa Restaurant,Spanish Restaurant,French Restaurant,Argentinian Restaurant,Asian Restaurant,Bakery,Bar
11,Roosevelt Island,Latin American Restaurant,Arepa Restaurant,Churrascaria,French Restaurant,Food Truck,Empanada Restaurant,Deli / Bodega,Cuban Restaurant,Caribbean Restaurant,Jazz Club


* CLUSTER 6

In [59]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 6, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Chelsea,Latin American Restaurant,Peruvian Restaurant,Argentinian Restaurant,Arepa Restaurant,Bakery,Brazilian Restaurant,French Restaurant,Caribbean Restaurant,Cuban Restaurant,Mexican Restaurant
22,Little Italy,Latin American Restaurant,Arepa Restaurant,Argentinian Restaurant,Peruvian Restaurant,South American Restaurant,Brazilian Restaurant,Mexican Restaurant,Tapas Restaurant,Venezuelan Restaurant,Deli / Bodega
27,Gramercy,Latin American Restaurant,Argentinian Restaurant,Brazilian Restaurant,Mexican Restaurant,Arepa Restaurant,Nightclub,Empanada Restaurant,Deli / Bodega,Peruvian Restaurant,Food Truck
34,Sutton Place,Latin American Restaurant,Empanada Restaurant,Juice Bar,Argentinian Restaurant,Food Truck,Jazz Club,Arepa Restaurant,Seafood Restaurant,Puerto Rican Restaurant,Steakhouse
38,Flatiron,Latin American Restaurant,Arepa Restaurant,Argentinian Restaurant,Brazilian Restaurant,Mexican Restaurant,Caribbean Restaurant,Empanada Restaurant,Deli / Bodega,Cuban Restaurant,Nightclub


* CLUSTER 7

In [60]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 7, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Latin American Restaurant,Arepa Restaurant,Puerto Rican Restaurant,Mexican Restaurant,Café,Peruvian Restaurant,Bar,Argentinian Restaurant,Brazilian Restaurant,Bakery
9,Yorkville,Latin American Restaurant,Arepa Restaurant,Peruvian Restaurant,Argentinian Restaurant,Food Truck,Empanada Restaurant,Deli / Bodega,Cuban Restaurant,Churrascaria,Caribbean Restaurant
30,Carnegie Hill,Latin American Restaurant,Peruvian Restaurant,Arepa Restaurant,Argentinian Restaurant,Food Truck,Empanada Restaurant,Deli / Bodega,Cuban Restaurant,Churrascaria,Caribbean Restaurant


* CLUSTER 8

In [61]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 8, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Latin American Restaurant,Empanada Restaurant,South American Restaurant,Seafood Restaurant,Caribbean Restaurant,Arepa Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Bakery
5,Manhattanville,Latin American Restaurant,Food Truck,Spanish Restaurant,Empanada Restaurant,Wine Bar,Caribbean Restaurant,Deli / Bodega,Cuban Restaurant,Churrascaria,Café
6,Central Harlem,Latin American Restaurant,Food Truck,Spanish Restaurant,Empanada Restaurant,Wine Bar,Caribbean Restaurant,Deli / Bodega,Cuban Restaurant,Churrascaria,Café


* CLUSTER 9

In [62]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 9, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Battery Park City,Latin American Restaurant,Arepa Restaurant,Brazilian Restaurant,Empanada Restaurant,Middle Eastern Restaurant,Peruvian Restaurant,South American Restaurant,Argentinian Restaurant,Bar,Café
29,Financial District,Latin American Restaurant,Peruvian Restaurant,Argentinian Restaurant,Empanada Restaurant,South American Restaurant,Brazilian Restaurant,Middle Eastern Restaurant,Churrascaria,Deli / Bodega,Cuban Restaurant
